In [6]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [7]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-15 08:32:08--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.77MB/s    in 0.5s    

2021-05-15 08:32:09 (1.77 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_AWS_Book").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [9]:
# Configuration for RDS instance
mode="overwrite"
jdbc_url = "jdbc:postgresql://big-data-challenge.csldkrhc63yg.ap-southeast-2.rds.amazonaws.com:5432/big_data_db"
config = {"user":"postgres",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [10]:
from pyspark import SparkFiles
#load software reviews data from s3.amazonaws.com
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)

book_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), inferSchema=True, dateFormat="yyyy-MM-dd",sep="\t")
book_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

# **Drop duplicates and incomplete rows**

In [11]:
print(book_df.count())
book_df = book_df.dropna()
print(book_df.count())
book_df = book_df.dropDuplicates()
print(book_df.count())

3105520
3105360
3105360


# **Examine the schema**

In [12]:
book_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# **Transfor Dataframe type**

In [13]:
import datetime
book_df = book_df.withColumn("review_date",book_df["review_date"].cast("date"))
book_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53066271|R1001W6J9AVY60|0060194839|     496072800|Volumetrics: Feel...|           Books|          4|            7|         13|   N|                N|        Good Book...|I bought this boo...| 2001-05-23|
|         US|   49255792|R100JJ4ITUS5NK|0375760628|     447916207|Slave to Fashion:...|           Books|          4|    

In [14]:
book_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



# **Create Dataframe for review_id_table**

In [15]:
review_id_df_new = book_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df_new.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1001W6J9AVY60|   53066271|0060194839|     496072800| 2001-05-23|
|R100JJ4ITUS5NK|   49255792|0375760628|     447916207| 2004-04-13|
|R100M49CK52349|   49394632|0553380966|     199085075| 2003-09-17|
|R100MJQ2PCB17R|   34342725|1570719276|     875642269| 2003-06-23|
|R100MQANUTTHLB|   41458372|1561840580|     557383629| 2003-12-23|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [16]:
review_id_df_pre = spark.read.jdbc(url=jdbc_url,table="review_id_table", properties=config)
review_id_df = review_id_df_new.union(review_id_df_pre)

In [17]:
print(review_id_df.count())
review_id_df = review_id_df.dropDuplicates(["review_id"])
print(review_id_df.count())

9325975
9325975


# **Create Dataframe for products_table**

In [18]:
products_df_new = book_df.select(["product_id", "product_title"])
products_df_new.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0060194839|Volumetrics: Feel...|
|0375760628|Slave to Fashion:...|
|0553380966|The Diamond Age: ...|
|1570719276|Living La Dolce V...|
|1561840580|Pacts With the De...|
+----------+--------------------+
only showing top 5 rows



In [19]:
products_df_pre = spark.read.jdbc(url=jdbc_url,table="products", properties=config)
products_df=products_df_new.union(products_df_pre)

In [20]:
print(products_df.count())
products_df = products_df.dropDuplicates(["product_id"])
print(products_df.count())

4180091
1854444


# **Create Dataframe for customers_table**

In [21]:
customers_df_new = book_df.select(["customer_id"])
customers_df_new = customers_df_new.groupBy("customer_id").count()
customers_df_new.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   51163715|    6|
|   12134452|    1|
|   52953209|  110|
|   50926345|    1|
|   17235720|    2|
+-----------+-----+
only showing top 5 rows



In [22]:
customers_df_pre = spark.read.jdbc(url=jdbc_url,table="customers", properties=config)
if customers_df_pre.count():
  customers_df=customers_df_new.join(customers_df_pre,on=["customer_id"],how="outer")
  customers_df=customers_df.fillna(0)
  customers_df=customers_df.withColumn('total', sum(customers_df[col] for col in ["count", "customer_count"]))
  customers_df = customers_df.drop("count", "customer_count")
  customers_df = customers_df.withColumnRenamed("total", "customer_count")
  customers_df.show(5)
else:
  customers_df = customers_df_new.withColumnRenamed("count", "customer_count")
  customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      10206|             1|
|      12940|             3|
|      13289|             1|
|      13832|             1|
|      16574|             1|
+-----------+--------------+
only showing top 5 rows



# **Create Dataframe for vine_table**

In [23]:
vine_df_new = book_df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df_new.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1001W6J9AVY60|          4|            7|         13|   N|
|R100JJ4ITUS5NK|          4|            1|          1|   N|
|R100M49CK52349|          5|            2|          4|   N|
|R100MJQ2PCB17R|          5|            9|         14|   N|
|R100MQANUTTHLB|          5|            6|         10|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [24]:
vine_df_pre = spark.read.jdbc(url=jdbc_url,table="vine_table", properties=config)
vine_df = vine_df_new.union(vine_df_pre)

In [25]:
print(vine_df.count())
vine_df = vine_df.dropDuplicates(["review_id"])
print(vine_df.count())

9325975
9325975


# **Write DataFrame to RDS**

In [26]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [27]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [28]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [29]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)